In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!cat exp/nb_01_matmul.py


##########################################
###### This file was autogenerated. ######
######### DO NOT EDIT this file. #########
##########################################
### file to edit: dev_nb/imflash217__01_matmul.ipynb ####

from exp.nb_00_exports import *
import operator

def test(a, b, cmp, cname=None):
    if cname is None:
        cname = cmp.__name__
    assert cmp(a, b), f"{cname}:\n{a}\n{b}"

def test_eq(a,b):
    test(a, b, operator.eq, "==")

from pathlib import Path
from IPython.core.debugger import set_trace
import gzip

import matplotlib as mpl
import matplotlib.pyplot as plt

import pickle
import math
import torch
from fastai import datasets

MNIST_URL = "http://deeplearning.net/data/mnist/mnist.pkl"


### test whether the results from two methods are same or not
### as its difficult to accurately achieve the same numbers due to floating-point operation
### we will create a function to test whether two tensor (of same shape) are
### APPROXIMATELY EQUAL withing some 

## The `forward()` & `backward()` passes

In [3]:
#export
from exp.nb_01_matmul import *

def get_data(url=MNIST_URL):
    path = datasets.download_data(url, ext=".gz")
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return map(torch.tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, mean, std):
    return (x-mean)/std               ## using broadcasting to normalize

```
Step-1: Get the dataset.
```

In [4]:
### Step-1: getting the datset
x_train, y_train, x_valid, y_valid = get_data(url=MNIST_URL)

In [5]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

(torch.Size([50000, 784]),
 torch.Size([50000]),
 torch.Size([10000, 784]),
 torch.Size([10000]))

In [6]:
mean_train = x_train.mean()
std_train  = x_train.std()

(mean_train, std_train)

(tensor(0.1304), tensor(0.3073))

```
Step-2: Normalize your dataset properly
        * Normalize the valid/test dataset with the stats of train data only.
```

In [7]:
### Step-2: Normalizing the train dataset
x_train = normalize(x_train, mean_train, std_train)

### Step-2: * Normalize the valid dataset with the stast of train data
x_valid = normalize(x_valid, mean_train, std_train)

In [8]:
### Checking the mean & std afetr normalization
(x_train.mean(), x_train.std(), x_valid.mean(), x_valid.std())

(tensor(3.0614e-05), tensor(1.), tensor(-0.0058), tensor(0.9924))

In [9]:
### We can see above that the mean and std of the train/valid data are now around 0 & 1 respectively
### So, we can now create a function to validate if the mean & std are within the limits 
### using our test methods from nb_01_matmul.py module

In [10]:
#export
def test_near_zero(x, tol=1e-3):
    assert x.abs() < tol, f"Near zero: {x}"

In [11]:
### checking the mean & std of normalized train data
test_near_zero(x=x_train.mean())
test_near_zero(x=(1-x_train.std()))

In [12]:
### checking the mean of noramlized valid data
test_near_zero(x=x_valid.mean())

AssertionError: Near zero: -0.005817127414047718

In [13]:
### checking the std of normalized valid data
test_near_zero(x=(1-x_valid.std()))

AssertionError: Near zero: 0.007566630840301514

In [14]:
n, m = x_train.shape          ### total number of rows & cols in the train data
c = y_train.max()+1           ### total number of label classes

print(n, m, c)

50000 784 tensor(10)


## Foundations version

### Basic architecture:

```
Step-3: Define the number of hidden layers and number of hidden cells in each layer
```

In [15]:
### Step-3: No of hidden units in 1 hidden layer
num_hidden_cells = 50

In [16]:
### Step-4: Define the weight and bias matrices
w1 = torch.randn(m, num_hidden_cells)
b1 = torch.randn(num_hidden_cells)
w2 = torch.randn(num_hidden_cells, 1)
b2 = torch.randn(1)

In [17]:
### this will be around (mean=0, std=1) as we sampled from torch.randn()
w1.mean(), w1.std()

(tensor(-0.0108), tensor(1.0004))

In [18]:
### this will be around (mean=0, std=1) as we sampled from torch.randn()
w2.mean(), w2.std()

(tensor(0.4142), tensor(0.9814))

In [19]:
# test_near_zero(w1.mean())
# test_near_zero(w2.mean())
# test_near_zero(1-w1.std())
# test_near_zero(1-w2.std())

In [35]:
def layer_linear(inputs, weights, bias):
    return inputs @ weights + bias

In [21]:
l1_out = layer_linear(inputs=x_valid, weights=w1, bias=b1)

In [22]:
l1_out.mean(), l1_out.std()

(tensor(1.5419), tensor(28.1979))

In [23]:
### the mean and std are clearly not as intended to be to (0 & 1)
### so, we need to initalize dour weights in a more proper manner to achieve zero-mean and unit-variance

### WE WILL USE KAIMING-HE INITIALIZATION

In [59]:
### applying kaiming initialization to our weights
w1 = torch.randn(m, num_hidden_cells)*math.sqrt(2/m)
b1 = torch.randn(num_hidden_cells)
w2 = torch.randn(num_hidden_cells, 1)*math.sqrt(2/num_hidden_cells)
b2 = torch.randn(1)

In [60]:
w1.mean(), w1.std()

(tensor(2.5482e-06), tensor(0.0507))

In [61]:
w2.mean(), w2.std()

(tensor(0.0332), tensor(0.2217))

In [62]:
### we will now use these weights in linear layer and see the mean, std of outputs
l1_out_kaiming = layer_linear(inputs=x_valid, weights=w1, bias=b1)
l1_out_kaiming.mean(), l1_out_kaiming.std()

(tensor(0.0832), tensor(1.8792))

In [63]:
l2_out_kaiming = layer_linear(inputs=l1_out_kaiming, weights=w2, bias=b2)
l2_out_kaiming.mean(), l2_out_kaiming.std()

(tensor(-1.2827), tensor(1.6462))

```
Step-4: Defining the activations function ReLU
```

In [64]:
### Defining ReLU activation function
def relu(inputs:torch.Tensor):
    return inputs.clamp_min(0.)

def relu_improved(inputs:torch.Tensor):
    return inputs.clamp_min(0.) - 0.5

In [69]:
l1_out = relu(layer_linear(inputs=x_valid, weights=w1, bias=b1))

l1_out.mean(), l1_out.std()

(tensor(0.7883), tensor(1.1597))

In [70]:
l2_out = relu(layer_linear(inputs=l1_out, weights=w2, bias=b2))

l2_out.mean(), l2_out.std()

(tensor(1.0744), tensor(0.9116))

In [71]:
l1_imp = relu_improved(layer_linear(inputs=x_valid, weights=w1, bias=b1))
l1_imp.mean(), l1_imp.std()

(tensor(0.2883), tensor(1.1597))

In [72]:
l2_imp = relu_improved(layer_linear(inputs=l1_imp, weights=w2, bias=b2))
l2_imp.mean(), l2_imp.std()

(tensor(-0.0134), tensor(0.6988))

In [74]:
### As, till now we have implemented kaiming init ourselves
### We can now use the init module from pytorch

In [75]:
#export
from torch.nn import init

In [78]:
### using kaiming normal initialization from pytorch
w1 = torch.randn(m, num_hidden_cells)
b1 = torch.randn(num_hidden_cells)
init.kaiming_normal_(tensor=w1, mode="fan_out")

t = relu_improved(layer_linear(inputs=x_valid, weights=w1, bias=b1))

In [79]:
w1.mean(), w1.std()

(tensor(-0.0002), tensor(0.0509))

In [80]:
t.mean(), t.std()

(tensor(0.1764), tensor(1.0214))

In [81]:
w1.shape

torch.Size([784, 50])

In [82]:
import torch.nn

In [84]:
torch.nn.Linear(in_features=m, out_features=num_hidden_cells).weight.shape

torch.Size([50, 784])

In [87]:
### If you look closely you will see that the weight matrix used by pytorch is transposed
### So, that's the reason for using "fan_out" in kaiming init method

### How this is taken care of in the linear layer's matrix computation??
### In pytorch's linear layer, before matmul they transpose the weight matrix, thus validating the shapes for matrix computation

In [88]:
torch.nn.Linear??

In [91]:
torch.nn.Linear.reset_parameters??

In [92]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [93]:
def model(xb):
    layer1 = layer_linear(inputs=xb, weights=w1, bias=b1)
    layer2 = relu_improved(layer1)
    layer3 = layer_linear(inputs=layer2, weights=w2, bias=b2)
    return layer3

In [94]:
%timeit -n 10 _=model(x_valid)

9.84 ms ± 2.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [95]:
model(x_valid).shape

torch.Size([10000, 1])

In [96]:
y_valid.shape

torch.Size([10000])

In [97]:
x_valid.shape[0]

10000

In [98]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0], 1])

## Loss function: `mse()` : mean square function

In [115]:
#export
def mse(preds, target):
    """Mean Square Error loss"""
    return (preds.squeeze(-1)-target).pow(2).mean()

In [116]:
y_train = y_train.float()
y_valid = y_valid.float()

In [117]:
preds_train = model(x_train)
preds_train.shape, y_train.shape

(torch.Size([50000, 1]), torch.Size([50000]))

In [118]:
preds_valid = model(x_valid)
preds_valid.shape, y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [119]:
loss_train = mse(preds=preds_train, target=y_train)
loss_valid = mse(preds=preds_valid, target=y_valid)

loss_train, loss_valid

(tensor(18.2649), tensor(18.2924))

## Gradients and backward pass

In [128]:
def mse_grad(inp, targ):
    """
    Gradient of loss w.r.t output of the previous layer
    There are no learning parameters in MSE, so we need to find grad only wrt input'
    
        mse = (1/N)((y-y_hat)**2)
    so, mse_grad = 2*(1/N)(y-y_hat)    
    """
    inp.grad = 2. * (inp.squeeze(-1) - targ).unsqueeze(-1) / inp.shape[0]
    
    
def relu_grad(inp, out):
    """
    ReLU doesn't have any learnable parameters;
    so we need to find the gradient of output wrt input
    
        relu(x) = 0 if x < 0 else x
    so, relu_grad_x = 0 if x < 0 else 1
    
    NOTE: We need to also accomodate for the upstream gradient
    So, inp.grad = relu_grad_x * out.grad
    """
    inp.grad = (inp>0).float() * out.grad
    
    
def linear_grad(inp, out, weights, bias):
    """
    Linear layer has "weights" and "bias" as trainable parameters;
    so we need to find the gradients of the ouput wrt input, "weights" and "bias".
    
    linear(x, w, b) = y = x@w + b
    
    linear_grad_x = w
    linear_grad_w = x
    linear_grad_b = 1
    
    NOTE: x, w, b are matrices/vectors
    NOTE: Remember to account for the upstream gradients
    """
    inp.grad     = out.grad @ (weights.t())
    weights.grad = (inp.unsqueeze(-1) * out.grad.unsqueeze(1)).sum(dim=0)
    bias.grad    = out.grad.sum(dim=0)
    

In [129]:
def forward_and_backward(inp, targ):
    ### forward pass for 2-linear_layer model
    l1   = inp @ w1 + b1
    l2   = relu(l1)
    out  = l2 @ w2 + b2
    
    loss = mse(out, targ)        ### NOTE: The actual value of loss is not used in backward pass
    
    ### backward pass
    ### call the backward passes in reveresed order of layers
    
    mse_grad(inp=out, targ=targ)                        ### loss layer
    linear_grad(inp=l2, out=out, weights=w2, bias=b2)   ### 2nd linear layer
    relu_grad(inp=l1, out=l2)                           ### relu layer
    linear_grad(inp=inp, out=l1, weights=w1, bias=b1)   ### 1st linear layer
    

In [133]:
forward_and_backward(inp=x_valid, targ=y_valid)

In [139]:
### saving the gradients wrt inputs, weights & bias to test later against pytorch autograd module
inp_g = x_valid.grad.clone()
w1_g  = w1.grad.clone()
b1_g  = b1.grad.clone()
w2_g  = w2.grad.clone()
b2_g  = b2.grad.clone()

In [140]:
b1_g.shape, b1.shape

(torch.Size([50]), torch.Size([50]))

In [141]:
b2_g.shape, b2.shape

(torch.Size([1]), torch.Size([1]))

In [142]:
### using pytorch autograd to calculate our gradients wrt inputs, weights & bias
inp_pt = x_valid.clone().requires_grad_(True)
w1_pt  = w1.clone().requires_grad_(True)
b1_pt  = b1.clone().requires_grad_(True)
w2_pt  = w2.clone().requires_grad_(True)
b2_pt  = b2.clone().requires_grad_(True)

In [145]:
def forward(inp, targ):
    l1 = inp @ w1_pt + b1_pt
    l2 = relu(l1)
    out = l2 @ w2_pt + b2_pt
    
    loss = mse(out, targ)
    return loss

In [146]:
loss = forward(inp=inp_pt, targ=y_valid)

In [147]:
loss.backward()

In [149]:
test_near(inp_g, inp_pt.grad)
test_near(w1_g, w1_pt.grad)
test_near(b1_g, b1_pt.grad)
test_near(w2_g, w2_pt.grad)
test_near(b2_g, b2_pt.grad)

In [151]:
### As all above test cases passed....So we are doing almost as good as Pytorch
### SO NOW WE NEED TO re-FACTOR our implementataion for effieciency and usage

## Re-Factor :

### Type-1 re-factoring: 
```
LAYERS as CLASSES
```

In [207]:
class ReLU():
    """forward and backard passes for ReLU layer"""
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5      ### improved relu
        return self.out
    
    def backward(self):
        self.inp.grad = (self.inp > 0).float() * self.out.grad
        
        
class Linear():
    """forward and backward passes for linear layer"""
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias    = bias
        
    def __call__(self, inp):
        self.inp     = inp
        self.out     = self.inp @ self.weights + self.bias
        return self.out
    
    def backward(self):
        self.inp.grad     = (self.out.grad) @ self.weights.t()
        self.weights.grad = (self.inp.unsqueeze(-1) * self.out.grad.unsqueeze(1)).sum(dim=0)
        self.bias.grad    = (self.out.grad.sum(dim=0))
        
    
class MSE():
    """forward and backward passes for MSE loss"""
    def __call__(self, inp, targ):
        self.inp  = inp
        self.targ = targ
        self.out  = (self.inp.squeeze() - self.targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.grad = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / (self.inp.shape[0])


In [208]:
### now we have defined our classes for needed layers and parts
### we will build our model for 2-layer fully connected MSE classifier

class Model():
    """a 2 layer fully connected network with relu & mse-loss"""
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Linear(weights=w1, bias=b1), ReLU(), Linear(weights=w2, bias=b2)]
        self.loss = MSE()
        
    def __call__(self, inp, targ):
        for layer in self.layers:
            inp = layer(inp)
        return self.loss(inp=inp, targ=targ)
    
    def backward(self):
        self.loss.backward()                       ### calling the backward pass on loss layer (as it is not included in self.layers)
        for layer in reversed(self.layers):        ### calling the backward for each layer in reversed order
            layer.backward()
        

In [209]:
### resetting the gradients before calculating it; so that it can be properly tested
w1.grad, b1.grad, w2.grad, b2.grad = [None]*4

model = Model(w1=w1, b1=b1, w2=w2, b2=b2)

In [210]:
### calling the FORWARD PASS
%time loss = model(inp=x_valid, targ=y_valid)
# %timeit -n 10 loss = model(inp=x_valid, targ=y_valid)

CPU times: user 37.1 ms, sys: 667 µs, total: 37.7 ms
Wall time: 18.7 ms


In [211]:
### calling the BACKWARD PASS
%time model.backward()
# %timeit -n 10 model.backward()

CPU times: user 1.59 s, sys: 636 ms, total: 2.23 s
Wall time: 1.11 s


In [212]:
# test_near(inp_g, x_valid.grad)
test_near(w1_g, w1.grad)
test_near(b1_g, b1.grad)
test_near(w2_g, w2.grad)
test_near(b2_g, b2.grad)


AssertionError: near:
tensor([[-0.2645,  0.0257,  0.2722,  ...,  0.7142,  0.0177,  0.0084],
        [-0.2645,  0.0257,  0.2722,  ...,  0.7142,  0.0177,  0.0084],
        [-0.2645,  0.0257,  0.2722,  ...,  0.7142,  0.0177,  0.0084],
        ...,
        [-0.2645,  0.0257,  0.2722,  ...,  0.7142,  0.0177,  0.0084],
        [-0.2645,  0.0257,  0.2722,  ...,  0.7142,  0.0177,  0.0084],
        [-0.2645,  0.0257,  0.2722,  ...,  0.7142,  0.0177,  0.0084]])
tensor([[-0.3648,  0.0336,  0.3678,  ...,  0.9847,  0.0243,  0.0117],
        [-0.3648,  0.0336,  0.3678,  ...,  0.9847,  0.0243,  0.0117],
        [-0.3648,  0.0336,  0.3678,  ...,  0.9847,  0.0243,  0.0117],
        ...,
        [-0.3648,  0.0336,  0.3678,  ...,  0.9847,  0.0243,  0.0117],
        [-0.3648,  0.0336,  0.3678,  ...,  0.9847,  0.0243,  0.0117],
        [-0.3648,  0.0336,  0.3678,  ...,  0.9847,  0.0243,  0.0117]])